In [ ]:
import numpy as np

def mc_trinomial_asian_batched(
    S0, K, r, sigma, T, N,
    Ns_total,
    u, m, d, pu, pd,
    batch_size=100_000,
    seed=42,
    verbose=True,
):
    """
    Monte Carlo per Asian call con modello trinomial (down/mid/up),
    implementato in modo da reggere Ns_total molto grandi usando batching.

    Parametri
    ---------
    S0 : float
        Prezzo iniziale.
    K : float
        Strike dell'Asian call.
    r : float
        Tasso risk-free (continuously compounded).
    sigma : float
        Volatilità (non usata direttamente se passi già u, m, d).
    T : float
        Maturity.
    N : int
        Numero di step temporali.
    Ns_total : int
        Numero totale di percorsi Monte Carlo.
    u, m, d : float
        Fattori moltiplicativi per up, mid, down.
    pu, pd : float
        Probabilità di up e down. Quella di mid è 1 - pu - pd.
    batch_size : int
        Numero di percorsi simulati per batch.
        Deve essere abbastanza grande da sfruttare il vettoriale,
        ma piccolo abbastanza da stare in RAM.
    seed : int
        Seed del generatore random.
    verbose : bool
        Se True, stampa un po' di progresso sui batch.

    Ritorna
    -------
    price : float
        Stima Monte Carlo del prezzo dell'Asian call.
    stderr : float
        Errore standard sulla stima del prezzo (1 sigma).
    """

    # Controllo probabilità
    if pu < 0 or pd < 0 or pu + pd > 1 + 1e-12:
        raise ValueError("Probabilità inconsistente: deve valere pu >= 0, pd >= 0, pu + pd <= 1.")

    discount = np.exp(-r * T)
    rng = np.random.default_rng(seed)

    # Accumulatori globali (per payoff non scontati)
    global_mean = 0.0
    global_M2 = 0.0  # somma dei quadrati delle deviazioni dal mean (per varianza)
    global_n = 0     # numero totale di campioni processati

    n_batches = (Ns_total + batch_size - 1) // batch_size

    for b in range(n_batches):
        # Numero di percorsi in questo batch
        start = b * batch_size
        Ns_batch = min(batch_size, Ns_total - start)

        if verbose and (b == 0 or (b + 1) % max(1, n_batches // 10) == 0):
            print(f"[Batch {b+1}/{n_batches}] Simulo {Ns_batch} percorsi...")

        # Stato iniziale dei percorsi in questo batch
        S = np.full(Ns_batch, S0, dtype=float)
        sum_S = np.zeros(Ns_batch, dtype=float)

        # Evoluzione nel tempo
        for _ in range(N):
            # u_rand in [0,1)
            u_rand = rng.random(Ns_batch)

            # Aggiorna i prezzi usando solo operazioni vettoriali e senza array temporanei enormi:
            # 1. Tutti down: moltiplica per d
            S *= d

            # 2. Mid o up: per quelli con u_rand >= pd, moltiplica per (m/d)
            mask_mid_or_up = u_rand >= pd
            S[mask_mid_or_up] *= (m / d)

            # 3. Up: per quelli con u_rand >= 1 - pu, moltiplica per (u/m)
            mask_up = u_rand >= (1.0 - pu)
            S[mask_up] *= (u / m)

            # Aggiorna somma per media asiatica
            sum_S += S

        # Media aritmetica del prezzo per path
        average_price = sum_S / N

        # Payoff Asian call
        payoff_batch = np.maximum(average_price - K, 0.0)

        # Statistiche del batch
        batch_n = Ns_batch
        if batch_n > 1:
            batch_mean = payoff_batch.mean()
            batch_var = payoff_batch.var(ddof=1)
            batch_M2 = batch_var * (batch_n - 1)
        else:
            batch_mean = float(payoff_batch.mean())
            batch_M2 = 0.0

        # Combina con le statistiche globali usando la formula per unione di due campioni
        if global_n == 0:
            global_mean = batch_mean
            global_M2 = batch_M2
            global_n = batch_n
        else:
            delta = batch_mean - global_mean
            new_n = global_n + batch_n
            global_mean += delta * (batch_n / new_n)
            global_M2 += batch_M2 + delta * delta * (global_n * batch_n / new_n)
            global_n = new_n

    # Media e varianza globale dei payoff (non scontati)
    payoff_mean = global_mean
    if global_n > 1:
        payoff_var = global_M2 / (global_n - 1)
    else:
        payoff_var = 0.0

    # Prezzo e errore standard
    price = discount * payoff_mean
    stderr = discount * np.sqrt(payoff_var / global_n) if global_n > 0 else 0.0

    if verbose:
        print(f"\nDone. Ns_total = {Ns_total}")
        print(f"Payoff mean (undiscounted): {payoff_mean:.6g}")
        print(f"Payoff std  (undiscounted): {np.sqrt(payoff_var):.6g}")
        print(f"Asian call price:           {price:.6g}")
        print(f"Std error on price:         {stderr:.6g}")

    return price, stderr


In [17]:
import math

def haahtela_trinomial_step(r, sigma, dt, lam=1.12):
    """
    Parametri del trinomial tree di Haahtela per un singolo time step
    con volatilità costante (σ_max = σ).

    Input
    -----
    r   : tasso risk-free continuo (per anno, per esempio)
    sigma : volatilità istantanea (stessa unità temporale di r)
    dt  : lunghezza del time step
    lam : parametro di dispersione λ (>1), default ≈ 1.12 come suggerito da Haahtela

    Output (nell'ordine che volevi)
    ------
    m  : fattore centrale    = exp(r * dt)
    d  : fattore down
    u  : fattore up
    pm : prob. di andare al nodo centrale
    pu : prob. di up
    pd : prob. di down
    """

    # caso triviale: volatilità nulla → cammino deterministico
    if sigma == 0:
        m = math.exp(r * dt)
        u = d = m
        pu = pd = 0.0
        pm = 1.0
        return m, d, u, pm, pu, pd

    # 1) "volatilità discreta" corretta per la lognormale su Δt:
    #    σ̃^2 = e^{σ^2 Δt} - 1   (equazione (10) del paper)
    sigma_tilde_sq = math.exp(sigma**2 * dt) - 1.0
    sigma_tilde = math.sqrt(sigma_tilde_sq)

    # 2) fattore centrale (ramo "middle") – cresce al risk-free
    m = math.exp(r * dt)

    # 3) ampiezza dei salti up/down (equazioni (37)-(38)):
    #    u = m * exp(λ σ̃),  d = m * exp(-λ σ̃)
    k = lam * sigma_tilde
    u = m * math.exp(k)
    d = m * math.exp(-k)

    # 4) probabilità rischio-neutrali p_u, p_d, p_m
    #    determinate imponendo:
    #      - E[X]   = m
    #      - Var(X) = m^2 σ̃^2
    #
    #    con X ∈ {d, m, u} con prob. {p_d, p_m, p_u}.
    #
    #    Da questi vincoli (e p_u + p_m + p_d = 1) si ottiene:
    #       p_u = m^2 σ̃^2 / ((u - m) (u - d))
    #       p_d = m^2 σ̃^2 / ((m - d) (u - d))
    #       p_m = 1 - p_u - p_d

    denom_pu = (u - m) * (u - d)
    denom_pd = (m - d) * (u - d)

    pu = (m**2 * sigma_tilde_sq) / denom_pu
    pd = (m**2 * sigma_tilde_sq) / denom_pd
    pm = 1.0 - pu - pd

    # piccola correzione numerica per evitare robe tipo -1e-16
    eps = 1e-14
    pu = max(0.0, min(1.0, pu))
    pd = max(0.0, min(1.0, pd))
    pm = max(0.0, min(1.0, pm))

    return m, d, u, pm, pu, pd
S0 = 100.0
K = 100.0
r = 0.05
sigma = 0.2
T = 1.0
N = 15
dt= T / N

# parametri trinomiali (esempio)

u = 1.1
m = 1.0
d = 0.9
pu = 0.25
pd = 0.25

#m, d, u, pm, pu, pd = haahtela_trinomial_step(r, sigma, dt, lam=1.12)
Ns_total = 10**7       # prova prima con valori più piccoli!
batch_size = 200_000   # 2e5 paths per batch

price, stderr = mc_trinomial_asian_batched(
    S0, K, r, sigma, T, N,
    Ns_total,
    u, m, d, pu, pd,
    batch_size=batch_size,
    seed=42,
    verbose=True,
)


[Batch 1/50] Simulo 200000 percorsi...
[Batch 5/50] Simulo 200000 percorsi...
[Batch 10/50] Simulo 200000 percorsi...
[Batch 15/50] Simulo 200000 percorsi...
[Batch 20/50] Simulo 200000 percorsi...
[Batch 25/50] Simulo 200000 percorsi...
[Batch 30/50] Simulo 200000 percorsi...
[Batch 35/50] Simulo 200000 percorsi...
[Batch 40/50] Simulo 200000 percorsi...
[Batch 45/50] Simulo 200000 percorsi...
[Batch 50/50] Simulo 200000 percorsi...

Done. Ns_total = 10000000
Payoff mean (undiscounted): 6.65467
Payoff std  (undiscounted): 10.7432
Asian call price:           6.33011
Std error on price:         0.0032316


In [12]:
import time
import matplotlib.pyplot as plt

# Assumo che mc_trinomial_asian_batched sia già definita da qualche parte, ad esempio:
# from your_module import mc_trinomial_asian_batched

def benchmark_mc_pricing_error(
    S0, K, r, sigma, T,
    u, m, d, pu, pd,
    P_ref,
    N_list=(25, 30, 50),
    Ns_min=10_000,
    Ns_max=50_000_000,
    n_Ns=10,
    n_runs=10,
    batch_size=100_000,
    base_seed=1234,
):
    """
    Esegue il benchmark Monte Carlo:
    - per N in N_list;
    - per Ns in n_Ns valori equispaziati tra Ns_min e Ns_max;
    - per ogni coppia (N, Ns) esegue n_runs run indipendenti;
    - misura walltime e errore rispetto a P_ref[N].

    Ritorna:
        results: dict strutturato come
            results[N]['Ns']         -> array dei Ns usati
            results[N]['time_mean']  -> walltime medio per ogni Ns
            results[N]['err_median'] -> mediana degli errori per ogni Ns
            results[N]['err_std']    -> std degli errori per ogni Ns
    """

    # Valori di Ns equispaziati in scala lineare (come da tua richiesta)
    Ns_values = np.linspace(Ns_min, Ns_max, n_Ns, dtype=int)

    # Inizializza struttura risultati
    results = {
        N: {
            'Ns': Ns_values.copy(),
            'time_mean': [],
            'err_median': [],
            'err_std': [],
        }
        for N in N_list
    }

    # Triplo loop: su N, su Ns, su run
    for iN, N in enumerate(N_list):
        print(f"\n=== N = {N} ===")
        if N not in P_ref:
            raise ValueError(f"Manca P_ref per N={N} in P_ref.")

        price_ref = P_ref[N]

        for jNs, Ns in enumerate(Ns_values):
            print(f"  -> Ns = {Ns}  ({jNs+1}/{len(Ns_values)})")

            run_times = []
            errors = []

            for run in range(n_runs):
                # Seed diverso per ogni (N, Ns, run) per indipendenza
                seed = base_seed + run + 100 * jNs + 10_000 * iN

                t0 = time.perf_counter()
                price, stderr = mc_trinomial_asian_batched(
                    S0, K, r, sigma, T, N,
                    Ns_total=Ns,
                    u=u, m=m, d=d, pu=pu, pd=pd,
                    batch_size=batch_size,
                    seed=seed,
                    verbose=False,   # niente spam
                )
                t1 = time.perf_counter()

                run_times.append(t1 - t0)
                errors.append(price - price_ref)

            run_times = np.array(run_times, dtype=float)
            errors = np.array(errors, dtype=float)

            # Walltime medio per questo (N, Ns)
            time_mean = run_times.mean()

            # Mediana e std degli errori (campionaria, ddof=1)
            err_median = np.median(errors)
            err_std = errors.std(ddof=1) if len(errors) > 1 else 0.0

            results[N]['time_mean'].append(time_mean)
            results[N]['err_median'].append(err_median)
            results[N]['err_std'].append(err_std)

        # Converte le liste in array numpy
        results[N]['time_mean'] = np.array(results[N]['time_mean'], dtype=float)
        results[N]['err_median'] = np.array(results[N]['err_median'], dtype=float)
        results[N]['err_std'] = np.array(results[N]['err_std'], dtype=float)

    return results


def plot_mc_pricing_error(results):
    """
    Plot del grafico stile Fig. 3:
    - asse x: walltime medio (scala log)
    - asse y: pricing error (mediana ± std)
    - per ogni N due curve (mediana+std, mediana-std) con stesso colore.
    """

    fig, ax = plt.subplots(figsize=(8, 5))

    # Colori fissi per ogni N
    colors = {
        25: 'tab:blue',
        30: 'tab:orange',
        50: 'tab:green',
    }

    for N, data in results.items():
        t_mean = data['time_mean']
        err_med = data['err_median']
        err_std = data['err_std']
        color = colors.get(N, None)

        y_plus = err_med + err_std
        y_minus = err_med - err_std

        # Curva sopra lo zero (mediana + std)
        ax.plot(
            t_mean, y_plus,
            label=f"N={N} median+std",
            color=color,
            linestyle='-',
            marker='o',
            linewidth=1.5,
        )

        # Curva sotto lo zero (mediana - std)
        ax.plot(
            t_mean, y_minus,
            label=f"N={N} median-std",
            color=color,
            linestyle='--',
            marker='o',
            linewidth=1.5,
        )

    # Linea orizzontale a zero (errore nullo)
    ax.axhline(0.0, color='black', linewidth=1.0)

    ax.set_xscale('log')  # tipico per questi plot (walltime su più ordini di grandezza)
    ax.set_xlabel("Walltime medio per run [s]")
    ax.set_ylabel("Pricing error  (MC - reference)")
    ax.set_title("Monte Carlo pricing error vs walltime")

    ax.legend()
    ax.grid(True, which='both', linestyle=':', linewidth=0.7, alpha=0.7)
    fig.tight_layout()
    plt.show()


# Esempio di uso (da adattare con i tuoi parametri reali)
if __name__ == "__main__":
    # Parametri di esempio
    S0 = 100.0
    K = 100.0
    r = 0.05
    sigma = 0.2
    T = 1.0

    # Parametri trinomiali (esempio!)
    u = 1.1
    m = 1.0
    d = 0.9
    pu = 0.25
    pd = 0.25

    # Prezzi "veri" / di riferimento (devi metterli tu!)
    P_ref = {
        25: 7.98985,  # rimpiazza con il valore "vero" per N=25
        30: 8.69942,  # idem
        50: 11.0844,  # idem
    }

    results = benchmark_mc_pricing_error(
        S0, K, r, sigma, T,
        u, m, d, pu, pd,
        P_ref,
        N_list=(25, 30, 50),
        Ns_min=10_000,
        Ns_max=1_000_000,
        n_Ns=10,
        n_runs=10,
        batch_size=100_000,
        base_seed=1234,
    )

    plot_mc_pricing_error(results)



=== N = 25 ===
  -> Ns = 10000  (1/10)
  -> Ns = 120000  (2/10)
  -> Ns = 230000  (3/10)
  -> Ns = 340000  (4/10)
  -> Ns = 450000  (5/10)
  -> Ns = 560000  (6/10)
  -> Ns = 670000  (7/10)
  -> Ns = 780000  (8/10)
  -> Ns = 890000  (9/10)


KeyboardInterrupt: 